In [1]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import FuncFormatter

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment
from defdap.quat import Quat

from pathlib import Path

import copy 
import pandas as pd
import datetime

from scipy.signal import find_peaks
from scipy.linalg import sqrtm, polar




import os

# get dictools stuff 
import sys
# sys.path.append("c:/work/hrdic-tools/")
# import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

In [2]:
exp = experiment.Experiment()

# load DIC data 
data_dir = Path('.')
dic_frame = experiment.Frame()
for dic_file in sorted(data_dir.glob('map_*.txt')):
    hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')

hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth

for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.set_crop(left=100,right=100,top=100,bottom=100)
    # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
    print(dic_map)

Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)


In [3]:
ebsd_frame = experiment.Frame()
ebsd.Map(data_dir / 'Post_EBSD/map_small.cpr',
         increment=exp.increments[0], frame=ebsd_frame)

Loaded EBSD data (dimensions: 1825 x 1750 pixels, step size: 0.2 um)


In [4]:
ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.set_homog_point()

dic_map = exp.increments[0].maps['hrdic']
# dic_map.set_homog_point(vmin=0,vmax=0.1)

In [5]:
# dic_map = exp.increments[0].maps['hrdic']
# dic_frame.homog_points = [(952, 825),
#  (1506, 586),
#  (1597, 69),
#  (858, 167),
#  (1143, 700),
#  (1205, 191),
#  (1769, 553),
#  (1273, 1097),
#  (1539, 1171),
#  (1761, 1666),
#  (1109, 1381),
#  (1565, 1618),
#  (82, 358),
#  (304, 563),
#  (635, 671),
#  (923, 538),
#  (927, 1116),
#  (235, 1302),
#  (871, 1446),
#  (96, 1743)]

# ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_frame.homog_points = [(1946, 1565),
#  (2375, 1384),
#  (2443, 999),
#  (1877, 1079),
#  (2094, 1469),
#  (2142, 1091),
#  (2574, 1359),
#  (2193, 1760),
#  (2396, 1823),
#  (2571, 2196),
#  (2065, 1989),
#  (2418, 2156),
#  (1279, 1218),
#  (1454, 1369),
#  (1700, 1457),
#  (1925, 1345),
#  (1929, 1788),
#  (1391, 1934),
#  (1881, 2037),
#  (1281, 2274)]

In [6]:
ebsd_map = exp.increments[0].maps['ebsd']
ebsd_frame.homog_points = [(274, 381),
 (891, 265),
 (1613, 265),
 (958, 760),
 (1148, 1241),
 (1425, 1035),
 (378, 1109),
 (390, 1409),
 (1615, 1407),
 (889, 1245),
 (710, 635),
 (576, 558),
 (556, 696),
 (318, 703),
 (545, 499),
 (1216, 958),
 (1375, 752),
 (781, 976),
 (1303, 732),
 (1061, 746),
 (1514, 594),
 (1274, 510),
 (1435, 427),
 (1117, 664),
 (425, 1075),
 (631, 1400),
 (631, 1036),
 (769, 1126),
 (1152, 942),
 (1240, 1339),
 (1528, 1149),
 (1084, 1193)]


dic_frame.homog_points = [(83, 363),
 (856, 170),
 (1759, 146),
 (949, 829),
 (1186, 1442),
 (1536, 1173),
 (233, 1304),
 (245, 1687),
 (1761, 1668),
 (868, 1444),
 (634, 672),
 (465, 576),
 (444, 757),
 (144, 785),
 (425, 495),
 (1273, 1087),
 (1470, 805),
 (727, 1117),
 (1382, 780),
 (1074, 814),
 (1643, 593),
 (1341, 480),
 (1543, 362),
 (1147, 700),
 (290, 1256),
 (548, 1663),
 (545, 1195),
 (719, 1310),
 (1194, 1069),
 (1303, 1594),
 (1654, 1333),
 (1109, 1382)]

In [7]:
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    



    dic_map.plot_map(
        'max_shear', vmin=0, vmax=0.10, 
        plot_scale_bar=False, plot_gbs='line'
    )
    plt.tight_layout()

Finished building quaternion array (0:00:07) 
Finished finding grain boundaries (0:00:20) 


In [8]:
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.plot_map(
        'r_ang'
    )

In [9]:
# find special boundaries 
misori_twin = Quat.from_axis_angle([1, 1, 1], 60*np.pi/180)
misori_twin_tol = 5*np.pi/180

# create all symmetric equivalent misorientations
misori_twin_all = []
syms = ebsd_map.primary_phase.crystal_structure.symmetries
for sym_i in syms:
    for sym_j in syms:
        misori_twin_all.append(sym_i.conjugate * misori_twin * sym_j)


# get rid of any duplicates
misori_twin_all = list(set(misori_twin_all))

# calculate neighbour network
ebsd_map.build_neighbour_network()

# loop over all grain boundary segments and check if the misorientation between
# the two grains is within tolerance of the twin misorientation
# store all gbs pairs with misorientation for later us 

all_gb_info = []
twin_gb_info = []
twin_lines = []

for grain1, grain2, b_seg in ebsd_map.neighbour_network.edges.data('boundary'):
    twin = False

    # calculate grain ref orientation
    grain1.calc_average_ori()
    grain2.calc_average_ori()

    misori = grain2.ref_ori * grain1.ref_ori.conjugate

    # # calculate misorientation angle in degrees 
    misori_ang = 2*np.arccos(grain1.ref_ori.mis_ori(grain2.ref_ori,ebsd_map.crystal_sym))*180/np.pi

    # add to list of everything 
    all_gb_info.append([grain1,grain2,misori,misori_ang,b_seg])

    # check if twin and add to twin list
    for misori_twin in misori_twin_all:
        if 2 * np.arccos(misori_twin.dot(misori)) < misori_twin_tol:
            twin = True
            break
    
    if not twin:
        continue
    
    twin_lines.append(b_seg)
    twin_gb_info.append([grain1,grain2,misori,misori_ang,b_seg])




# make into boundary set
s_bounds = ebsd.BoundarySet.from_boundary_segments(twin_lines)

Finished finding grains (0:00:05) twork..
Finished constructing neighbour network (0:00:07) 


In [10]:
# plot misorientation distribution function 

# sanity check for all twin boundaries having a MO ~ 60 degrees 
twin_mo_angle = []
for i in range(len(twin_gb_info)):
    twin_mo_angle.append(twin_gb_info[i][3])



all_mo_angle = []
for i in range(len(all_gb_info)):
    all_mo_angle.append(all_gb_info[i][3])

fig,ax = plt.subplots()
ax.hist(all_mo_angle,range=(0,65),bins=50,density=True)
ax.set_ylabel('Probability density / -')
ax.set_xlabel('Misorientation angle / °')
plt.tight_layout()

Mask individual boundaries to extract max shear for each

In [12]:
# functions

def mask_and_extract(dic_map,mask):
    # masks DIC map data BUT not in the DIC map object
    # extracts the data first, then masks, then returns the data 

    # max shear
    ems = copy.deepcopy(dic_map.data['max_shear'])
    ems[mask] = np.nan

    # strain components 
    e = copy.deepcopy(dic_map.data['e'])
    e[:,:,mask] = np.nan

    # rotations 
    r_ang = copy.deepcopy(dic_map.data['r_ang'])
    r_ang[mask] = np.nan

    return ems, e, r_ang

def make_log_log_histogram(fig,ax,data_list,bin_range,bins,density,colour_list):
    bin_list = np.logspace(np.log10(bin_range[0]),np.log10(bin_range[-1]),bins)

    for i in range(0,len(data_list)):

        hist = np.histogram(data_list[i].flatten(), bins=bin_list, density=density)

        y_vals = hist[0]
        x_vals = 0.5 * (hist[1][1:] + hist[1][:-1])

        ax.plot(x_vals,y_vals,'.',mfc=colour_list[i],mec=colour_list[i],ms=5)

    return  fig, ax 

In [13]:
# loop over every gb in dic map, extract misorientation, Ems, E, r angle 

dilation = 5

dic_gb_mori     = []
dic_gb_ems      = []
dic_gb_e        = []
dic_gb_r_ang    = []


for i in range(len(all_gb_info)):
    this_b_seg = ebsd.BoundarySet.from_boundary_segments([all_gb_info[i][4]])
    this_b_seg_dic = hrdic.BoundarySet.from_ebsd_boundaries(dic_map,this_b_seg)

    # check if this gb is in the DIC ROI 
    if np.all(this_b_seg_dic.image) == False: 
        dic_map.generate_threshold_mask(this_b_seg_dic.image,dilation=dilation,preview=False)

        this_mask = ~dic_map.mask

        this_ems, this_e, this_r_ang = mask_and_extract(dic_map,this_mask)

        dic_gb_mori.append(all_gb_info[i][3])
        dic_gb_ems.append(this_ems)
        dic_gb_e.append(this_e)
        dic_gb_r_ang.append(this_r_ang)

Filtering will remove 234 \ 3996001 (0.006 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 540 \ 3996001 (0.014 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 182 \ 3996001 (0.005 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 259 \ 3996001 (0.006 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 171 \ 3996001 (0.004 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data


KeyboardInterrupt: 

In [14]:
dic_gb_ems_mean = []
dic_gb_r_ang_mean = []
for i in range(len(dic_gb_ems)):
    dic_gb_ems_mean.append(np.nanmean(dic_gb_ems[i]))
    dic_gb_r_ang_mean.append(np.nanmean(dic_gb_r_ang[i]))

dic_gb_ems_mean = np.asarray(dic_gb_ems_mean)
dic_gb_r_ang_mean = np.asarray(dic_gb_r_ang_mean)

In [15]:
dic_gb_mori = np.asarray(dic_gb_mori)

In [ ]:
# plot 2d histogram to visualise the joint distribution

fig,ax = plt.subplots()
ax.hist2d(dic_gb_mori,dic_gb_r_ang_mean,bins=100,range=[[0,65],[-5,5]],density=True)
ax.set_xlabel('Grain boundary misorientation / ° ')
ax.set_ylabel('In-plane rotation angle / ° ')


Text(0, 0.5, 'In-plane rotation angle / ° ')

In [ ]:
# get sigma3 boundaries 
s_bounds_dic = hrdic.BoundarySet.from_ebsd_boundaries(dic_map,s_bounds)
dilation = 5

#segment strain map by boundaries type object

# sigma three boundaries
dic_map.generate_threshold_mask(s_bounds_dic.image,dilation=dilation,preview=False)
sb_mask = ~dic_map.mask

# all boundaries 
dic_map.generate_threshold_mask(dic_map.data['grain_boundaries'].image,dilation=dilation,preview=False)
gb_mask = ~dic_map.mask

# normal high angle gbs 
ha_mask = ~(gb_mask ^ sb_mask)

# grain cores 
gc_mask = ~gb_mask


# make some colours for the different areas for consistency 
colour_list = ['orangered',     # S3 boundaries
               'chartreuse',    # All boundaries
               'deepskyblue',   # HA grain boundaries
               'fuchsia'        # Grain cores 
               ]

In [ ]:
plt.figure()
plt.imshow(ems_gc,vmin=0,vmax=0.1)

In [ ]:
# sigma3 boundaries
ems_sb, e_sb, r_ang_sb = mask_and_extract(dic_map,sb_mask)

# all grain boundaries
ems_gb, e_gb, r_ang_gb = mask_and_extract(dic_map,gb_mask)

# high angle grain boundaries
ems_ha, e_ha, r_ang_ha = mask_and_extract(dic_map,ha_mask)

# grain cores
ems_gc, e_gc, r_ang_gc = mask_and_extract(dic_map,gc_mask)


# plt.imshow(r_ang_sb,vmin=-5,vmax=5,cmap='RdBu_r')

In [ ]:
fig,ax = plt.subplots()
fig, ax = make_log_log_histogram(fig,ax,[ems_sb,ems_ha,ems_gb,ems_gc],bin_range=[0.01,0.5],bins=100,density=True,colour_list=colour_list)
ax.set_xscale('log')
ax.set_yscale('log')
ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.legend(['S3GBs','HAGBs','All GBs','Grain cores'])
ax.set_xlabel('Max shear / - ')
ax.set_ylabel('Relative frequency')

In [ ]:
fig,ax = plt.subplots()
fig, ax = make_log_log_histogram(fig,ax,[abs(r_ang_sb),abs(r_ang_ha),abs(r_ang_gb),abs(r_ang_gc)],bin_range=[0.1,10],bins=100,density=True,colour_list=colour_list)
ax.set_xscale('log')
ax.set_yscale('log')
ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.legend(['S3GBs','HAGBs','All GBs','Grain cores'])
ax.set_xlabel('In plane rotation magnitude / degrees')
ax.set_ylabel('Relative frequency')

In [ ]:
fig,ax = plt.subplots()
fig, ax = make_log_log_histogram(fig,ax,[abs(e_sb[0,0]),abs(e_ha[0,0]),abs(e_gb[0,0]),abs(e_gc[0,0])],bin_range=[0.01,0.5],bins=100,density=True,colour_list=colour_list)
ax.set_xscale('log')
ax.set_yscale('log')
ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.legend(['S3GBs','HAGBs','All GBs','Grain cores'])
ax.set_xlabel('E11 / - ')
ax.set_ylabel('Relative frequency')

In [ ]:
plot = dic_map.plot_map('max_shear',vmin=0,vmax=0.1,plot_gbs=False,dilate_boundaries=True,plot_scale_bar=True)
plt.savefig('max_shear.png',dpi=500)
# plt.figure()
plot = dic_map.plot_map('max_shear',vmin=0,vmax=0.1,plot_gbs=False,dilate_boundaries=True,plot_scale_bar=True)
plot.add_grain_boundaries(kind='line',boundaries=s_bounds_dic,color='limegreen')
plt.savefig('max_shear_s3gbs.png',dpi=500)

In [ ]:
# test with a single map to make things easier
# exp = experiment.Experiment()

dic_dir = Path('./DIC/')
ebsd_pre_dir = Path('./Pre_EBSD/')
ebsd_post_dir = Path('./Post_EBSD/')

dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
dic_map = hrdic.Map(dic_dir/ 'map_44.txt', data_type='openpiv')

hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth


# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)

dic_map.set_scale(pixelsize)
dic_map.set_crop(left=100,right=100,top=100,bottom=100)

ebsd_pre_frame = experiment.Frame()
ebsd_map=ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

ebsd_map.find_grains()
ebsd_map.build_neighbour_network()

In [ ]:
# exp = experiment.Experiment()

# dic_dir = Path('./DIC/')
# ebsd_pre_dir = Path('./Pre_EBSD/')
# ebsd_post_dir = Path('./Post_EBSD/')

# dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
# hfw = 20.0 # microns
# pixelwidth = 2048
# pixelsize = hfw/pixelwidth

# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)
    
# ebsd_pre_frame = experiment.Frame()
# ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

# ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.find_boundaries()
# ebsd_map.find_grains
# ebsd_map.build_neighbour_network()

In [ ]:
# ebsd_post_frame = experiment.Frame()
# ebsd.Map(ebsd_post_dir / 'map_small.cpr',increment=exp.increments[-1], frame=ebsd_post_frame)

In [ ]:
exp.increments[1].maps['ebsd']

In [ ]:
# dic_map = exp.increments[0].maps['hrdic']
dic_map.set_homog_point(vmin=0,vmax=0.2)

# ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.set_homog_point()

# dic_map.homog_points = [(1722, 1177),
#  (1335, 1313),
#  (948, 1745),
#  (549, 1661),
#  (245, 1680),
#  (233, 1300),
#  (579, 1158),
#  (443, 756),
#  (559, 136),
#  (83, 359),
#  (1438, 495),
#  (857, 165)]

# ebsd_map.homog_points = [(2541, 1823),
#  (2240, 1932),
#  (1940, 2266),
#  (1631, 2207),
#  (1394, 2225),
#  (1388, 1933),
#  (1659, 1820),
#  (1554, 1519),
#  (1648, 1054),
#  (1279, 1219),
#  (2323, 1319),
#  (1875, 1080)]

# dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
# dic_map.plot_map('max_shear', vmin=0,vmax=0.1,plot_scale_bar=True,plot_gbs=True)
#  # link maps
# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_crop
#     dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
#     # dic_map.plot_map(
#     #     'max_shear', vmin=0, vmax=0.10, 
#     #     plot_scale_bar=False, plot_gbs='line'
#     # )
    # plt.tight_layout()

In [ ]:
# find twin boundaries
misori_twin = Quat.from_axis_angle([1, 1, 1], 60*np.pi/180)
misori_twin_tol = 5*np.pi/180

# create all symmetric equivalent misorientations
misori_twin_all = []
syms = ebsd_map.primary_phase.crystal_structure.symmetries
for sym_i in syms:
    for sym_j in syms:
        misori_twin_all.append(sym_i.conjugate * misori_twin * sym_j)
# get rid of any duplicates
misori_twin_all = list(set(misori_twin_all))

# loop over all grain boundary segments and check if the misorientation between
# the two grains is within tolerance of the twin misorientation
twin_lines = []
for grain1, grain2, b_seg in ebsd_map.neighbour_network.edges.data('boundary'):
    twin = False
    misori = grain2.ref_ori * grain1.ref_ori.conjugate
    for misori_twin in misori_twin_all:
        if 2 * np.arccos(misori_twin.dot(misori)) < misori_twin_tol:
            twin = True
            break
    
    if not twin:
        continue
    
    twin_lines += b_seg.boundary_lines

In [ ]:
ebsd_map.build_neighbour_network()

In [ ]:
ebsd_map.

In [ ]:
t = ebsd_map.neighbour_network.edges.data('boundary')

In [ ]:
plot = ebsd_map.plot_ipf_map([1,0,0])
plot.add_grain_boundaries(kind='line',boundaries=twin_lines, colour='red')

In [ ]:
 # plot maps
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_crop
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    # dic_map.plot_map(
    #     'max_shear', vmin=0, vmax=0.10, 
    #     plot_scale_bar=False, plot_gbs='line'
    # )
    # plt.tight_layout()

dic_map = exp.increments[-1].maps['hrdic'] 
dic_map.plot_map(
    
)